In [ ]:
https://medium.com/@srinipotnuru/06d17152b51b

In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader

######### Content Processing Block ###############################

## Loading PDF file from local file directory
## read the content and store it in data object 
local_path = "./alphabet/Praveen_13Yrs_Datascience_AI.pdf"

if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
else:
    print("Upload a PDF file for processing.")

print(data[0].page_content)

c:\Users\LENOVO\Desktop\ItsAIWork\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Praveen Kumar V – Data science, AI – 13 plus Yrs – IIIT Hyderabad

Phone: 9663045588 email: praveenv8ai@gmail.com LinkedIn: https://www.linkedin.com/in/praveen-kumar-62a95212/

Executive Summary

Highly skilled data science professional, leader, adept at translating business problems into actionable data-driven solutions. Proficient to apply respective skills to deliver innovative scalable and impactful results that align with organizational goals.

Exploring potential Analytics Leadership roles that leverage experience gained across – diverse areas in

Business Transformation • Pharma Mfg, eCommerce, D2C

• People Excellence & Team management

Innovation, Strategic Thinking

Business Leadership Communication • Stakeholder Success Enablement

Analytics Technical and Leadership Competencies

Hands on Execution • Proficient Analytical, Problem-solving skills • Product Management, vision implementation • Data Engineering and pipeline development • Data Visulisation and communication

Anal

In [2]:
## Converting content into dense vector embeddings 
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma 

#Split and chunk the data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)


# Add the chunks to vector database, which takes the model for creating the embeddings.
vector_db = Chroma.from_documents(
                                    documents=chunks, 
                                    embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
                                    collection_name="local-rag"
                                )
###################################################

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_25780\1743440145.py:14: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
OllamaEmbeddings: 100%|██████████| 1/1 [00:06<00:00,  6.63s/it]


In [3]:
######### Retrieval + Generation of Response ##############################
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

local_llm = "llama3.1"
llm = ChatOllama(model=local_llm)

QUERY_PROMPT = PromptTemplate(
    input_variables = ["question"],
    template="""You are an AI Language model assistant. Your task is to generate five different versions of the given user question to retrieve relavant documents from a vector databaase. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search. Provide these alternative questions separated by newlines. 
    Original question: {question} """
)


retriever = MultiQueryRetriever.from_llm(vector_db.as_retriever(),llm, prompt=QUERY_PROMPT)

# RAG Prompt
template = """Answer the question based ONLY on the following context: 
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt 
    | llm 
    | StrOutputParser()
)
q = "Give me the list of companies Paraveen worked for"
response = chain.invoke(q)

print(response)

###################################################

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_25780\1878178932.py:9: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=local_llm)
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.09s/it]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.13s/it]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.10s/it]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.

Here is the list of companies Praveen worked for:

1. Samsung R&D Institute (since June 2022)
2. Gramener (July 2019 to June 2022)
3. DataJango (Jan 2019 to July 2019)
4. Mphasis (Sep 2017 to Aug 2018)
5. Bridgei2i Analytics Solutions (May 2015 to Sep 2017)
6. IBM India (July 2011 to Feb 2015)
7. IIIT Hyderabad (as Assistant Mentor for MSIT program, for 4 Months in Year 2010)


In [ ]:
https://medium.com/@drjulija/how-i-built-a-basic-rag-for-pdf-qa-in-a-few-lines-of-python-code-9849c32e59f0

In [1]:
# import libraries
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
##rom langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOllama
#import ollama
from langchain_ollama import OllamaEmbeddings

In [2]:
MODEL_NAME = "llama3.2:1b"
llm = ChatOllama(model=MODEL_NAME)
result = llm.invoke("in a sentence, what is ocean")
print(result.content)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20740\2474310538.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=MODEL_NAME)


The ocean is the largest body of saltwater on Earth, covering over 70% of its surface and playing a crucial role in regulating the planet's climate, weather patterns, and supporting life on land and in the air.


In [2]:
DOC_PATH = "./alphabet/alp_data.pdf"
CHROMA_PATH = "my_chrm_db" 
EMBEDDING_MODEL = "nomic-embed-text"

In [3]:
# ----- Data Indexing Process -----

# load your pdf doc
loader = PyPDFLoader(DOC_PATH)
pages = loader.load()

# split the doc into smaller chunks i.e. chunk_size=500
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

# get OpenAI Embedding model
#embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
embeddings=OllamaEmbeddings(model=EMBEDDING_MODEL)

In [4]:
db_chroma = Chroma.from_documents(documents = chunks, embedding=embeddings, collection_name=CHROMA_PATH)

ValueError: Expected Embedings to be non-empty list or numpy array, got [] in upsert.

In [ ]:
# embed the chunks as vectors and load them into the database
#db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)

# ----- Retrieval and Generation Process -----

# this is an example of a user question (query)
query = 'what are the top risks mentioned in the document?'

# retrieve context - top 5 most relevant (closests) chunks to the query vector 
# (by default Langchain is using cosine distance metric)
docs_chroma = db_chroma.similarity_search_with_score(query, k=5)

# generate an answer based on given user query and retrieved context information
context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

ValueError: Expected Embedings to be non-empty list or numpy array, got [] in upsert.

In [ ]:
# you can use a prompt template
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.
Provide a detailed answer.
Don’t justify your answers.
Don’t give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""

# load retrieved context and user query in the prompt template
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)

In [ ]:
result = llm.invoke(prompt)
print(result.content)

In [ ]:
# call LLM model to generate the answer based on the given context and query
model = ChatOpenAI()
response_text = model.predict(prompt)

"""
Generated response:

The top risks mentioned in the provided context are:
1. Decline in the value of investments
2. Lack of adoption of products and services
3. Interference or interruption from various factors such as modifications, terrorist attacks, natural disasters, etc.
4. Compromised trade secrets and legal and financial risks
5. Reputational, financial, and regulatory exposure
6. Abuse of platforms and misuse of user data
7. Errors or vulnerabilities leading to service interruptions or failure
8. Risks associated with international operations.
"""